In [1]:
###Cargando los paquetes 
using JuMP, HiGHS, LinearAlgebra, Plots, Random, CSV, DataFrames, XLSX

In [2]:
#Cargamos los datos que se encuentran en la otra hoja del excel 
G=XLSX.readtable("DietProblemPollyModificado.xlsx","Data parte e")
### Tipo de G
println("El tipo de G es\n",typeof(G),"\n")
### G tiene dos componentes: G.data y G.column_labels
println("G.data es\n",G.data,"\n")
println("G.column_labels es\n",G.column_labels)

El tipo de G es
XLSX.DataTable



G.data es
Any[Any

["Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "],

 Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[12.0, 0.0, 1.0, 10.0, 39.0, 48.0, 60.0], Any[2.0, 14.0, 11.0, 4.0, 

12.0, 3.0, 60.0], Any[2000.0, 6000.0, 5000.0, 1000.0, 10000.0, 3000.0, missing], Any[4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing], Any[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, missing], Any[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, missing], Any[0.0, 0.0, 0.0, 0.0, 0.0,

 1.0, missing], Any[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, missing]]

G.column_labels es


[:Food, Symbol("Protein "), Symbol("Calcium "), :Carbohidratos, :Grasas, Symbol("Price per seving "), Symbol("Limit per day "), :Manuel, Symbol("Seba Lillo "), Symbol("Seba Fuenzalida"), Symbol("Alberto ")]


In [3]:
data = DataFrame(G);
display(data)

Row,Food,Protein,Calcium,Carbohidratos,Grasas,Price per seving,Limit per day,Manuel,Seba Lillo,Seba Fuenzalida,Alberto
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,12.0,2.0,2000.0,4.0,0.0,1.0,0.0,0.0
2,Chicken,32.0,12.0,0.0,14.0,6000.0,3.0,1.0,0.0,0.0,0.0
3,Eggs,13.0,54.0,1.0,11.0,5000.0,2.0,0.0,0.0,0.0,0.0
4,Whole milk,8.0,285.0,10.0,4.0,1000.0,8.0,0.0,0.0,0.0,0.0
5,Cherry pie,4.0,22.0,39.0,12.0,10000.0,2.0,0.0,0.0,0.0,1.0
6,Pork with beans,14.0,80.0,48.0,3.0,3000.0,2.0,0.0,0.0,1.0,0.0
7,Minimo,55.0,800.0,60.0,60.0,missing,missing,missing,missing,missing,missing


In [9]:
###Creando el modelo en conjunto con el solver a usar
modelo = Model(HiGHS.Optimizer) 
println(modelo)

n=6
m=3


Feasibility
Subject to


3

In [10]:
@variable(modelo, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#agregamos las restricciones de nutrientes
@constraint(modelo, protein[j in 2:5], sum(x[i]*data[i,j] for i in 1:6) >= data[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo, limit[i in 1:6],x[i]<=data[i,7])
#agregamos las restricciones de preferencia de comidas
@constraint(modelo, preferencia[j in 8:11], sum(x[i]*data[i,j] for i in 1:6)>=1)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 8:11
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 preferencia[8] : x[2] >= 1.0
 preferencia[9] : x[1] >= 1.0
 preferencia[10] : x[6] >= 1.0
 preferencia[11] : x[5] >= 1.0

In [11]:
@objective(modelo, Min, sum(x[i]*data[i,6] for i in 1:6))
println(modelo)

Min 2000 x[1] + 6000 x[2] + 5000 x[3] + 1000 x[4] + 10000 x[5] + 3000 x[6]
Subject to
 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 protein[4] : 12 x[1] + x[3] + 10 x[4] + 39 x[5] + 48 x[6] >= 60.0
 protein[5] : 2 x[1] + 14 x[2] + 11 x[3] + 4 x[4] + 12 x[5] + 3 x[6] >= 60.0
 preferencia[8] : x[2] >= 1.0
 preferencia[9] : x[1] >= 1.0
 preferencia[10] : x[6] >= 1.0
 preferencia[11] : x[5] >= 1.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0


 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[5] integer
 x[6] integer



In [12]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo)

println("Optimal value: ", objective_value(modelo))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
4 rows, 6 cols, 23 nonzeros
2 rows, 6 cols, 12 nonzeros
Objective function is integral with scale 0.001

Solving MIP model with:
   2 rows
   6 cols (2 binary, 4 integer, 0 implied int., 0 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   23000           inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      29000
  Dual bound        29000
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    29000 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
          

In [13]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))
end

El valor de x[1] es 1.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 8.0
El valor de x[5] es 1.0
El valor de x[6] es 1.0
